In [43]:
# The first part of this code utilizes with some modifications code by Sung Pil Moon from https://blog.nycdatascience.com/student-works/project-3-web-scraping-company-data-from-indeed-com-and-dice-com/
# load the library
from bs4 import BeautifulSoup as Soup
import urllib, requests, re, pandas as pd

# indeed.com url
base_url = 'http://www.indeed.com/jobs?q=data+scientist&jt=fulltime&sort='
sort_by = 'date'          # sort by data
start_from = '&start='    # start page number

pd.set_option('max_colwidth',500)    # to remove column limit (Otherwise, we'll lose some info)
df = pd.DataFrame()   # create a new data frame

# previously used range(1,101), temporarily changed to range(1,3) for fast testing
for page in range(1,3): # page from 1 to 100 (last page we can scrape is 100)
    page = (page-1) * 10  
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
    target = Soup(urllib.request.urlopen(url), "lxml") 

    targetElements = target.find_all('div', attrs={'class' : 'row result'}) # we're interested in each row (= each job)
    
    # trying to get each specific job information (such as company name, job title, urls, ...)
    for elem in targetElements:
        # Some of the stuff extracted with .getText() seems to throw an error, can come back and fix
        comp_name = None
        #comp_name = elem.find('span', attrs={'itemprop':'name'}).getText().strip()
        job_title = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        home_url = "http://www.indeed.com"
        job_link = "%s%s" % (home_url,elem.find('a').get('href'))
        job_addr = None
        #job_addr = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()
        job_posted = None
        #job_posted = elem.find('span', attrs={'class': 'date'}).getText()

        comp_link_overall = None
        #comp_link_overall = elem.find('span', attrs={'itemprop':'name'}).find('a')
        if comp_link_overall != None: # if company link exists, access it. Otherwise, skip.
            comp_link_overall = "%s%s" % (home_url, comp_link_overall.attrs['href'])
        else: comp_link_overall = None

        # add a job info to our data frame
        df = df.append({'comp_name': comp_name, 'job_title': job_title, 
                        'job_link': job_link, 'job_posted': job_posted,
                        'overall_link': comp_link_overall, 'job_location': job_addr,
                        'overall_rating': None, 'wl_bal_rating': None, 
                        'benefit_rating': None, 'jsecurity_rating': None, 
                        'mgmt_rating': None, 'culture_rating': None
                       }, ignore_index=True)
        
df.head()

,benefit_rating,comp_name,culture_rating,job_link,job_location,job_posted,job_title,jsecurity_rating,mgmt_rating,overall_link,overall_rating,wl_bal_rating
0,None,None,None,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Acjr3lGWLBzJnSLr0JOeIhnRJBIlNlnblRoY9-2yUkfIBGbbR8dCn46gaxPaD96dVUskQr8ZAv8EQttb0JU7O5RnHNmn-e-IPXKM4Bo-HYV4HqRdZN_rJqA3wiVXHQlh9evWNyZAhJPIYcDerTABnsWGSSnpr1vuLjYTlfiFUeb_9wSc2kqodhvb5z3pzsUR-0xaqq1ZSnZl7ckaS0K94pvzxGW-XAAVKsw_ZovPcFEJYCeiWPjCR-JhXeLNcQY6sdBd_-eCA5cILIkzhvBsDNFybQK4QEZEIwoVCbXuIHEw0FsoVb55w1Yi1f_LxgZPccUmZkMr8BGPhRoCdIEVb9wZLfcZYeMBE0fQEmwUIHEg9D59102joBJjlWWixKHxNgAYQbrQbYPD6ldzQzJg-_DPlyPYdB5Nflvkdlx6BtAssiEj6XFZfgL2soieM...,None,None,Modeling Analyst - Data Mining and Statistical Analysis,None,None,None,None,None
1,None,None,None,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dqrr-Rjg2XLksGNbp1zr5zrpCZUmaYUPA5zQ_vrCj_X6a_99tNJpNpXW6b2-NnzR-s72LGlIrkkvWFRwr2frmiwkGfTVc8Qgg6uKo4D4z8Hcb6niEHqdp_J0Q9g6aYSoYTmug_FVffV-BytzsyRywanOurAcHwufn3fWvupe99CW-LRr12S0zYUg8Iael0eglSflKZAV-7tXNrygLVXJTD_FD5AxA6OBwPn5XRmgbYVCw3JCk4NftNAk0UQiHKNrRIxYEYJpN0440wuwP1x99NNO6ukCe33_MvJ8Ax80rQ04yKi-_Z9uto3QzujgnWgt1eTgBK7YrQ-ANalMKc8M-Gqg82FYyngxBJ1Pxp7qDYHA_sCfL6gxG5Q9joSCCjxVaqLPWdMD011TBb2eAdsbBoIkXhI5D3_ic=&p=3&sk=&fvj=1,None,None,Data Scientist,None,None,None,None,None
2,None,None,None,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DPGvvdYJGqX8bSRQNtYnhbAlNNNTb3VzXRAjulw0epZeO9VAqYHIF81JX6ijIk9llV9QXVX5BkVwnobaRz-Q3KGMJZjmGbFFM90iUe6P1D3yiMLqvSBEvX9aCg-ykSXnKZ-EafNLrEdz7XJlrS-e64-xiblvC-4DVzMn7BadqyjHd3znRbaHH98AA8dXS1roTBFxvYCyVlwBsqZWpd7_1e12SvXMqPGP1qAq6E_PI3aZ9VXWACthWS5SfgyCKOn9DmXcLYDjUX-rEezspGynC_o9bUNsLbq4iCV1wI-a9mPkrn0oJr6Cm2fCedFaGOgNbiQP_dm-nXFet2ckVYeWvNQN5SlaBNWdZtYT3dM-idc8yrGnlgCyIC7gFjzfCggii_mLxwN3dcgfcyNLJcyvEauPq3VbhkvL9Q-muJDL5Ak6xODNb-UnSECZ7IQZD...,None,None,Commerical Data Scientist,None,None,None,None,None
3,None,None,None,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DPhTdUClI5LzfhAu1yKalG75v0eNLMlcd6maXcCfBUXB7GYGM2E5a_f7GNHa2Qt13lD7mA-aM8vH_br4gyFz-uZCWFI_0AzXjp2nlLcrEtqs1vRPzt4uih7EVwc2MXL3SIsjIY33fo_Ju-elGwbzHdhbIc4G__w1DdniiiDBIyCE_S2hE9dOpzIKUtbd9Ubfv_GapzoWVkWe-g9gmTSa2BJ7gY5Atfe17CBs8Sz52wZJzo1LtKHQ1WvS2at4Iyt8nAQqdxG88OOcyQRjVJ6bAv66e4payfSI_TlWMzixBMdVinpNADdmsG905N8r3vR9H3qtkmN7wqSw_B7elKgHCBzYSSCdvsbHz24b79DkAuKgcJ-EwewTWR0pDsae7DHiZR6tP2vHuMvg==&p=3&sk=&fvj=0,None,None,"Statistician, R, SAS-AML Scenario Development & Enhancement-Capital Markets",None,None,None,None,None
4,None,None,None,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AcKrCDQ_eChMN_hHoeVwk0t6ouhlvRsMi3bqSMrWgRh5dQMLvJv6BTSai9X1ceMGysF1dkT0kxx30u0tjXgGzgua0aC9J7MjgTMZMGax7CP_WLHlXD5K0ikNotvJmjv-VTQh0vrnnZl8RyXoyUxVlw4p8jWFIfX3LfQbV9x7pjyA9wlp_9_9o35BR8J6tdXx3S32jH8ocEiU_9ZPAsoI7LjoTxBo2ZCVTlHo0zwKT1voWHuTYu_RjOdgjLX0naJrUzqZd5u-jf6hCdlzJweWMOjhcb-6f8HQFjYdRQJ_U3AX7kP87YfzOKAmFhSalTG8SsGiTyMYL7DxDxg93CAU1FWQ3FdEBskywfcofA7c5bFgbCvmDasuAGIowZ9iAlQyiFlQTE5H6sV_76Ulw-OTh5rbkQTEAFYDRS8mftmTIDSE354S4dteWF_1r44WA...,None,None,Principal Data Scientist,None,None,None,None,None


In [46]:
test_job = Soup(urllib.request.urlopen(df["job_link"][0]), "lxml")
for elem in test_job.find_all(["script", "style"]):
    elem.extract()
    
print(df["job_link"][0], test_job.get_text())

http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Acjr3lGWLBzJnSLr0JOeIhnRJBIlNlnblRoY9-2yUkfIBGbbR8dCn46gaxPaD96dVUskQr8ZAv8EQttb0JU7O5RnHNmn-e-IPXKM4Bo-HYV4HqRdZN_rJqA3wiVXHQlh9evWNyZAhJPIYcDerTABnsWGSSnpr1vuLjYTlfiFUeb_9wSc2kqodhvb5z3pzsUR-0xaqq1ZSnZl7ckaS0K94pvzxGW-XAAVKsw_ZovPcFEJYCeiWPjCR-JhXeLNcQY6sdBd_-eCA5cILIkzhvBsDNFybQK4QEZEIwoVCbXuIHEw0FsoVb55w1Yi1f_LxgZPccUmZkMr8BGPhRoCdIEVb9wZLfcZYeMBE0fQEmwUIHEg9D59102joBJjlWWixKHxNgAYQbrQbYPD6ldzQzJg-_DPlyPYdB5Nflvkdlx6BtAssiEj6XFZfgL2soieM4VrP4NaLHuWNfRTYp2o-vaUgOAo_NxhU3-uneYdtBPdujgAxMSb4fRxaLLZdfFXUDdlY=&p=2&sk=&fvj=0 b"\n\n\n\n\n\nModeling Analyst - Data Mining and Statistical Analysis - GEICO Careers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\xe2\x98\xb0\n\t\t\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJobs\n\n\n\n\nHelp\n\n\n\n\n\n\n\n\n\nSettings\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tModeling Analyst - Data Mining and Statistical Analysis\n\t\t\t\t\n\n\n\nNew\n\n\